**Passo a Passo para o Bagging**
Preparação dos Dados

*Coletar e Limpar os Dados:* Garanta que os dados estão organizados e sem erros. Remova dados ausentes, normalize ou padronize as variáveis e separe os dados em conjuntos de treinamento e teste.

**Criação de Subconjuntos de Dados**

*Amostragem com Reposição:* Crie várias versões menores do seu conjunto de dados original. Faça isso selecionando aleatoriamente dados com reposição (ou seja, dados repetidos são permitidos). O número de subconjuntos depende de quantos modelos você quer treinar.

**Treinamento de Modelos**

Treinar Modelos Base: *texto em itálico* Treine um modelo simples (como uma árvore de decisão) em cada subconjunto de dados. Cada modelo aprende algo diferente, pois os subconjuntos são únicos.

**Combinação de Previsões**

*Unir as Previsões:* Depois de treinar todos os modelos, use todos para fazer previsões. Combine as previsões para obter um resultado final. Em classificações, use a maioria dos votos; em regressões, tire a média das previsões.

**Avaliação do Modelo**

Avaliar o Desempenho: *texto em itálico* Verifique a precisão do modelo combinado no conjunto de teste. Compare os resultados com um único modelo para ver as melhorias com o Bagging.

In [ ]:
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go
from scipy.stats import mode

# Carregar o conjunto de dados
iris = load_iris()
X, y = iris.data, iris.target

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fase de Bootstrap: Criação de subconjuntos de dados
n_samples = len(X_train)
subsets = [np.random.choice(n_samples, n_samples, replace=True) for _ in range(10)]

# Treinar os Modelos em cada subconjunto
classifiers = []
accuracies = []

for indices in subsets:
    X_subset, y_subset = X_train[indices], y_train[indices]
    clf = DecisionTreeClassifier()
    clf.fit(X_subset, y_subset)
    classifiers.append(clf)
    # Avaliar a precisão de cada modelo base
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Agregação das Previsões
predictions = np.zeros((len(X_test), len(classifiers)))
for i, clf in enumerate(classifiers):
    predictions[:, i] = clf.predict(X_test)

# Votação majoritária (para classificação)
final_predictions = mode(predictions, axis=1)[0].flatten()

# Avaliar o desempenho do modelo combinado
final_accuracy = accuracy_score(y_test, final_predictions)
print(f'Acurácia do modelo Bagging: {final_accuracy:.2f}')

# Criar gráfico interativo com plotly
fig = go.Figure()

# Adicionar barras para as precisões dos modelos base
fig.add_trace(go.Bar(
    x=[f'Modelo {i+1}' for i in range(len(accuracies))],
    y=accuracies,
    name='Precisão dos Modelos Base'
))

# Adicionar linha para a precisão do modelo combinado
fig.add_trace(go.Scatter(
    x=[f'Modelo {i+1}' for i in range(len(accuracies))],
    y=[final_accuracy] * len(accuracies),
    mode='lines+markers',
    name='Precisão do Modelo Bagging'
))

# Configurar o layout do gráfico
fig.update_layout(
    title='Precisão dos Modelos Base vs. Modelo Bagging',
    xaxis_title='Modelos',
    yaxis_title='Precisão',
    showlegend=True
)

# Mostrar o gráfico
fig.show()


Acurácia do modelo Bagging: 1.00


Bagging, ou Bootstrap Aggregating, é uma técnica que uso para melhorar a precisão de modelos de aprendizado de máquina.

Eu aplico Bagging quando quero reduzir a variância do modelo, tornando-o mais estável e menos sensível a flutuações nos dados de treinamento. Além disso, essa técnica me ajuda a evitar overfitting, garantindo que o modelo não se ajuste demais aos dados. Outra vantagem é a melhora na precisão, pois a combinação de vários modelos gera previsões mais confiáveis.

Geralmente, uso Bagging em modelos como árvores de decisão, que podem variar bastante. Também recorro a essa abordagem quando trabalho com dados complexos ou quando preciso treinar vários modelos simples e rápidos. Além disso, é útil para prevenir overfitting, especialmente quando estou lidando com conjuntos de dados pequenos.

Um ótimo exemplo de Bagging é o Random Forest, que combina múltiplas árvores de decisão para gerar previsões mais precisas.